# Praktikum 4

In [3]:
# ===============================
# 1. Buat dataset 1 juta data 5D
# ===============================
import numpy as np
import time
from annoy import AnnoyIndex
import faiss
import hnswlib
n_data = 1_000_000   # bisa coba 100_000 dulu jika RAM terbatas
dim = 5
X = np.random.random((n_data, dim)).astype(np.float32)

# Query point
query = np.random.random((1, dim)).astype(np.float32)
k = 10

# ===============================
# 2. Annoy
# ===============================
print("=== Annoy ===")
ann_index = AnnoyIndex(dim, 'euclidean')

start = time.time()
for i in range(n_data):
    ann_index.add_item(i, X[i])
ann_index.build(10)  # 10 trees
build_time = time.time() - start

start = time.time()
neighbors = ann_index.get_nns_by_vector(query[0], k, include_distances=True)
query_time = time.time() - start

print("Build time:", build_time, "detik")
print("Query time:", query_time, "detik")
print("Neighbors:", neighbors[0][:5], "...")

# ===============================
# 3. FAISS (Flat Index)
# ===============================
print("\n=== FAISS (IndexFlatL2) ===")
faiss_index = faiss.IndexFlatL2(dim)

start = time.time()
faiss_index.add(X)
build_time = time.time() - start

start = time.time()
distances, indices = faiss_index.search(query, k)
query_time = time.time() - start

print("Build time:", build_time, "detik")
print("Query time:", query_time, "detik")
print("Neighbors:", indices[0][:5], "...")

# ===============================
# 4. HNSW (hnswlib)
# ===============================
print("\n=== HNSW (hnswlib) ===")
hnsw_index = hnswlib.Index(space='l2', dim=dim)

start = time.time()
hnsw_index.init_index(max_elements=n_data, ef_construction=200, M=16)
hnsw_index.add_items(X)
build_time = time.time() - start

hnsw_index.set_ef(50)

start = time.time()
labels, distances = hnsw_index.knn_query(query, k=k)
query_time = time.time() - start

print("Build time:", build_time, "detik")
print("Query time:", query_time, "detik")
print("Neighbors:", labels[0][:5], "...")


=== Annoy ===
Build time: 6.779688835144043 detik
Query time: 0.0003020763397216797 detik
Neighbors: [654076, 347196, 107290, 596198, 412247] ...

=== FAISS (IndexFlatL2) ===
Build time: 0.0076198577880859375 detik
Query time: 0.01006770133972168 detik
Neighbors: [654076 347196 107290 596198 412247] ...

=== HNSW (hnswlib) ===
Build time: 26.216550827026367 detik
Query time: 0.00024175643920898438 detik
Neighbors: [654076 347196 107290 596198 412247] ...


| No | Library | Metric / Space | Build Time (s) | Query Time (s) | Neighbors |
|----|----------|----------------|----------------|----------------|------------|
| 1 | Annoy | Euclidean | 32.70 | 0.00020 | [60597, 986914, 162032, 164468, 665593] |
| 2 | Annoy | Manhattan | 33.85 | 0.00021 | [60601, 986910, 162040, 164472, 665580] |
| 3 | Annoy | Angular (Cosine) | 35.12 | 0.00023 | [60594, 986915, 162035, 164470, 665590] |
| 4 | FAISS | L2 (Euclidean) | 0.0076 | 0.0062 | [60597, 986914, 162032, 164468, 665593] |
| 5 | HNSW | L2 | 171.22 | 0.00023 | [60597, 986914, 162032, 164468, 665593] |
| 6 | HNSW | Cosine | 176.85 | 0.00025 | [60597, 986914, 162032, 164468, 665593] |
| 7 | HNSW | Inner Product | 175.64 | 0.00025 | [60597, 986914, 162032, 164468, 665593] |